In [1]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras as extras

In [2]:
param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [3]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

conn = connect(param_dic)
cursor = conn. cursor()

Connecting to the PostgreSQL database...
Connection succesful


# Parte 1 - SQL

1. 
    a) ¿Puedes describirnos el procedimiento que has seguido para cargar los tres ficheros?

1.    
    b) ¿Puedes compartirnos la definición de la tabla que estás usando para contestar las
preguntas planteadas?

2. Una vez cargados los tres ficheros en tu base de datos, ¿cuántos registros tiene la
tabla? Compártenos la SQL que has usado.

In [4]:
table = 'yellowtaxis'

row_count = f'''
    SELECT COUNT(vendorid)
    FROM {table};
'''

cursor.execute(row_count)
result = cursor.fetchone();

print()
print(f"La tabla tiene {result[0]} registros")
print()


La tabla tiene 15711654 registros



3. Para cada uno de los tres meses con datos ¿podrías decirnos cuál es el trayecto más
corto, más largo y el trayecto medio?, en distancia y en tiempo. Compártenos la
sentencia SQL que has usado.


In [ ]:
def execute(query):
    
    cursor.execute(query)
    result = cursor.fetchone()[0]
    
    return result

        January

In [19]:
longest_trip_time = f'''
    SELECT (tpep_dropoff_datetime - tpep_pickup_datetime) AS trip_time
    FROM yellowtaxis
    WHERE month = 'January'
    ORDER BY trip_time DESC
    LIMIT 1;
'''

execute(longest_trip_time)

datetime.timedelta(days=5, seconds=79507)

In [31]:
shortest_trip_time = f'''
    SELECT (tpep_dropoff_datetime - tpep_pickup_datetime) AS trip_time
    FROM yellowtaxis
    WHERE month = 'January'
    ORDER BY trip_time ASC
    LIMIT 1;
    
'''
execute(shortest_trip_time)

datetime.timedelta(days=-2, seconds=6578)

In [26]:
6578/60

109.63333333333334

In [6]:
def trip(month, meassure, tipo):
    ms = ['distancia', 'tiempo']
    t = ['corto', 'largo', 'medio']
    assert meassure  in ms, f"Specify measure from: {ms}"
    assert tipo in t, f"Specify type from: {t}"
    time = 'tpep_pickup_datetime - tpep_dropoff_datetime'
    #distance = 
    query = f'''
        SELECT ({}) AS trip_{}
        FROM yellowtaxis
        WHERE month = '{}'
        ORDER BY trip_time {}
        LIMIT 1;'''
    
    if meassure == 'tiempo':
        if tipo == 'largo':
            query.format(time, 'time', month,'DESC')
        elif tipo == 'corto':
            query.format(time, 'time', 'ASC')
        else:
            query = f'''
                    SELECT AVG({time}) AS trip_time_avg
                    FROM yellowtaxis
                ;
            '''
    else:
        if tipo == 'largo':
            query.format('trip_distance', 'distance', 'DESC')
        elif tipo == 'corto':
            query.format('trip_distance', 'distance', 'ASC')
        else:
            query = f'''
                    SELECT AVG(trip_distance) AS trip_distance_avg
                    FROM yellowtaxis
                ;
            '''            
    cursor.execute(query)
    result = cursor.fetchone()
    
    return result

SyntaxError: f-string: empty expression not allowed (<ipython-input-6-8018f671106e>, line 13)

In [ ]:
def execute(query):
    
    cursor.execute(query)
    result = cursor.fetchone()
    
    return result

In [ ]:
trip('January', 'tiempo', 'largo')

In [ ]:
def longest_commute(month, tipo):
    query = f'''
    SELECT (tpep_pickup_datetime - tpep_dropoff_datetime) AS trip_time
    FROM yellowtaxis
    ORDER BY trip_time DESC
    LIMIT 1
;
    '''
    pass

In [ ]:
def average_commute(month, tipo):
    query = f'''
    
    '''
    pass

4. Para cada uno de los tres meses con datos ¿podrías decirnos cuánta es la variación
porcentual en el número de servicios registrados con respecto al mes anterior? Por
ejemplo, en esta pregunta nos gustaría obtener un resultado así. Compártenos la
sentencia SQL que has usado

In [51]:
import pandas as pd
import datetime

df = pd.DataFrame({'timestamp':['2015-04-01 23:49:58','2016-04-02 23:49:58']})
df['timestamp'] = pd.to_datetime(df['timestamp'])  
print (df)

df['time1'] = df['timestamp'].dt.hour * 3600 + \
              df['timestamp'].dt.minute * 60 + \
              df['timestamp'].dt.second

df['time2'] = (pd.to_timedelta(df['timestamp'].dt.strftime('%Y:%m:%d %H:%M:%S'))
                 .dt.total_seconds().astype(int))
print (df)


            timestamp
0 2015-04-01 23:49:58
1 2016-04-02 23:49:58


ValueError: expected hh:mm:ss format

In [82]:
import pandas as pd
import datetime as dt

df = pd.DataFrame({'timestamp':['2015-04-01 23:49:58','2016-04-02 23:49:58']})
df['timestamp'] = pd.to_datetime(df['timestamp'])  
print (df)



            timestamp
0 2015-04-01 23:49:58
1 2016-04-02 23:49:58


In [83]:
df.dtypes

timestamp    datetime64[ns]
dtype: object

In [84]:
df['time'] = df['timestamp'].dt.total_seconds()

AttributeError: 'DatetimeProperties' object has no attribute 'total_seconds'

In [48]:
diff = df['time1'][1] - df['time1'][0]
print(diff)

0


In [100]:
df = pd.DataFrame({'date1':pd.date_range('2015-01-01 ', periods=3),
                   'date2':pd.date_range('2015-01-01 02:00:00', periods=3, freq='23H'),
                   'date3':pd.date_range('2015-01-03 03:00:00', periods=3, freq='23H') 
                  })

print (df.dtypes)

date1    datetime64[ns]
date2    datetime64[ns]
date3    datetime64[ns]
dtype: object


In [101]:
df

,date1,date2,date3
0,2015-01-01,2015-01-01 02:00:00,2015-01-03 03:00:00
1,2015-01-02,2015-01-02 01:00:00,2015-01-04 02:00:00
2,2015-01-03,2015-01-03 00:00:00,2015-01-05 01:00:00


In [102]:
df['diff'] = df['date3'] - df['date3']
df['seconds'] = df['diff'] .dt.total_seconds()

In [103]:
df

,date1,date2,date3,diff,seconds
0,2015-01-01,2015-01-01 02:00:00,2015-01-03 03:00:00,0 days,0.0
1,2015-01-02,2015-01-02 01:00:00,2015-01-04 02:00:00,0 days,0.0
2,2015-01-03,2015-01-03 00:00:00,2015-01-05 01:00:00,0 days,0.0
